In [1]:
import base64
import datetime
import json
import os
import random
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

pd.options.display.max_colwidth = 1000

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# Load the Data

In [2]:

# days_since_last_manager_activity	days since manager's last listing creation

In [3]:
raw_data = pd.read_json('tmp/raw_data.json', convert_dates=['created'])
#d = pd.read_json('tmp/raw_data.json')
raw_data.index.name = 'id'
d = pd.DataFrame()
print len(d)

0


In [4]:
d.loc[:, 'price_per_bedrooms'] = raw_data['price'] / (0.001 + raw_data['bedrooms'])
d.loc[:, 'rooms_diff'] = raw_data['bedrooms'] - raw_data['bathrooms'] + 0.0
d.loc[:, 'rooms_sum'] = raw_data['bedrooms'] + raw_data['bathrooms'] + 0.0
d.loc[:, 'bathrooms_per_bedrooms'] = raw_data['bathrooms'] / (raw_data['bedrooms'] + 0.001)
d.head()

,price_per_bedrooms,rooms_diff,rooms_sum,bathrooms_per_bedrooms
id,,,,
0,2947.052947,0.0,2.0,0.999001
1,1424.287856,1.0,3.0,0.499750
10,999.666778,1.5,4.5,0.499833
100,3754.245754,0.0,2.0,0.999001
1000,1649.175412,1.0,3.0,0.499750


In [5]:
timediff = (raw_data.created.max() - raw_data.created.min()).components
max_diff_hour = timediff.days * 24 + timediff.hours
print max_diff_hour

2135


In [6]:
def sort_list(l):
    l.sort()
    return l

managers_dates = raw_data.groupby('manager_id')['created'].apply(list).map(sort_list).to_dict()


In [7]:
(pd.Timestamp('2016-04-24 01:24:19') - pd.Timestamp('2016-04-24 01:11:26')).components.hours

0

In [8]:
def time_since_managers_last_listing(row):
    listing_dates = managers_dates[row['manager_id']]
    created = row['created']
    idx = listing_dates.index(created)
    if idx == 0:
        return np.nan
    else:
        timediff = (listing_dates[idx] - listing_dates[idx - 1]).components
        return 24 * timediff.days + timediff.hours
d.loc[:, 'hours_since_managers_last_listing'] = (
    raw_data[['manager_id', 'created']]
    .apply(lambda row: time_since_managers_last_listing(row), axis=1)).astype(float64)

In [9]:
d.dtypes

price_per_bedrooms                   float64
rooms_diff                           float64
rooms_sum                            float64
bathrooms_per_bedrooms               float64
hours_since_managers_last_listing    float64
dtype: object

In [10]:
d.head()

,price_per_bedrooms,rooms_diff,rooms_sum,bathrooms_per_bedrooms,hours_since_managers_last_listing
id,,,,,
0,2947.052947,0.0,2.0,0.999001,47.0
1,1424.287856,1.0,3.0,0.499750,52.0
10,999.666778,1.5,4.5,0.499833,0.0
100,3754.245754,0.0,2.0,0.999001,601.0
1000,1649.175412,1.0,3.0,0.499750,146.0


# Save the Data

In [11]:
d.to_csv('tmp/features_others.csv')